In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import pandas as pd
import seaborn as sns
import datetime as dt
import cpi
import requests


In [2]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

In [93]:
from thefuzz import fuzz
from thefuzz import process


In [3]:
# Right after importing seaborn (could also use 'whitegrid')
sns.set_theme(style='whitegrid', context='talk')

In [4]:
# surpress scientific notation
pd.options.display.float_format = '{:.2f}'.format

# Importing Data

## Lobster Export Data

### Which codes do relate to lobsters?

From : https://www.ic.gc.ca/app/scr/tdst/tdo/crtr.html?grouped=INDIVIDUAL&searchType=KS_CS&naArea=9999&countryList=ALL&toFromCountry=CDN&reportType=TE&timePeriod=5%7CComplete+Years&currency=CDN&productType=HS6&hSelectedCodes=%7C10600%7C30611%7C30612%7C30615%7C30621%7C30622%7C30625%7C30631%7C30632%7C30634%7C30691%7C30692%7C30694%7C160530&runReport=true

Searched for all codes containing word "lobster"

In [5]:
# jlab
path_1605 = 'data/full_data/1605_88Jan_to_23Sep_ATP_PV_Monthly.csv'
df_1605_full = pd.read_csv(path_1605, engine = 'python', skipfooter = 3,index_col=False, header = 1)
df_1605_full.head()

,Period,Commodity,Province,Country,State,Value ($),Quantity,Unit of measure
0,2013-04-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,195111,29802,Weight in kilograms
1,2013-12-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,285427,35966,Weight in kilograms
2,2017-10-01,"1605.59.00 - Molluscs, prepared or preserved, nes",Newfoundland and Labrador,United States,California,185609,22500,Weight in kilograms
3,2014-08-01,"1605.10.91 - Crab, snow (Queen), prepared or p...",Newfoundland and Labrador,United States,Alabama,191769,14969,Weight in kilograms
4,2013-01-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,170470,16053,Weight in kilograms


In [6]:
path_0306 = 'data/full_data/0306-Crust-Jan88-to-Sep23-prov.csv'
df_0306_full = pd.read_csv(path_0306, engine = 'python', skipfooter = 3,index_col=False, header = 1)
df_0306_full.head()

,Period,Commodity,Province,Country,State,Value ($),Quantity,Unit of measure
0,2014-06-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,220915,66528,Weight in kilograms
1,2023-04-01,"0306.14.10 - Crabs, snow (Queen), frozen",Newfoundland and Labrador,United States,California,1111067,61916,Weight in kilograms
2,2015-01-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,753558,174576,Weight in kilograms
3,2015-06-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,883171,199392,Weight in kilograms
4,2015-07-01,"0306.17.10 - Shrimps and prawns, frozen, in sh...",Newfoundland and Labrador,Saint Pierre and Miquelon,NaN,704,45,Weight in kilograms


## Country GDP

In [9]:
# TODO store API key as env variable 
api_key = 'DsPAQbicFL8WYvYihOWoFQ==Vd8zztny3V9w3Q7n'

In [10]:
gdp_data = pd.read_csv('data/gdp-data/gdp_data.csv',engine= 'python', header = 2)
gdp_data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2791061452.51,2963128491.62,2983798882.68,3092178770.95,3276187709.50,3395793854.75,2610038938.55,3126019385.47,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,21125015452.20,21616228139.04,23506279899.97,28048360188.28,25920665259.89,29472103269.74,...,1006526390112.83,927348469903.11,885176429223.51,1021043200767.01,1007196197587.41,1000834077088.55,927593321647.66,1081998139192.47,1169483734191.46,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,537777811.11,548888895.56,546666677.78,751111191.11,800000044.44,1006666637.78,...,20550582746.84,19998143635.87,18019554403.45,18896353155.88,18418860354.42,18904502222.21,20143451705.75,14583135236.57,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,10447637852.92,11173212080.39,11990534017.87,12727688165.45,13898109284.02,14929792387.52,...,894322482190.28,768644740597.12,691363412188.46,684898755570.72,767025741475.04,822538393591.45,786460035395.03,844459722152.99,877863284874.07,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,137244439121.37,87219300384.51,49840491178.15,68972769395.63,77792944471.95,69309110145.77,50241368243.63,65685435100.50,106713618735.43,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7074392884.05,6295845465.36,6682674062.96,7180768692.26,7878762812.20,7899741278.94,7717143395.26,9412034299.23,9429156201.96,NaN
262,"Yemen, Rep.",YEM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,43228585321.33,42444489460.94,31317824906.44,26842231204.80,21606161066.21,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,GDP (current US$),NY.GDP.MKTP.CD,8748596504.06,9225996313.29,9813996078.33,10854195662.66,11955995222.38,13068994777.63,...,381198869776.11,346709790458.56,323585509674.48,381448814653.46,404158882459.02,388531226582.35,337619569570.51,419015636064.71,405869718462.34,NaN
264,Zambia,ZMB,GDP (current US$),NY.GDP.MKTP.CD,713000000.00,696285714.29,693142857.14,718714285.71,839428571.43,1082857142.86,...,27141023558.08,21251216798.78,20958412538.31,25873601260.84,26311506435.06,23308667781.23,18110638619.37,22147649568.61,29784454055.94,NaN


In [11]:
gdp_data.drop(columns= ['Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)



In [12]:
gdp_data

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2791061452.51,2963128491.62,2983798882.68,3092178770.95,3276187709.50,3395793854.75,2610038938.55,3126019385.47,NaN,NaN
1,Africa Eastern and Southern,21125015452.20,21616228139.04,23506279899.97,28048360188.28,25920665259.89,29472103269.74,32014368121.37,33269509510.46,36327785494.93,...,1006526390112.83,927348469903.11,885176429223.51,1021043200767.01,1007196197587.41,1000834077088.55,927593321647.66,1081998139192.47,1169483734191.46,NaN
2,Afghanistan,537777811.11,548888895.56,546666677.78,751111191.11,800000044.44,1006666637.78,1399999966.67,1673333417.78,1373333366.67,...,20550582746.84,19998143635.87,18019554403.45,18896353155.88,18418860354.42,18904502222.21,20143451705.75,14583135236.57,NaN,NaN
3,Africa Western and Central,10447637852.92,11173212080.39,11990534017.87,12727688165.45,13898109284.02,14929792387.52,15910837741.97,14510579888.72,14968235781.96,...,894322482190.28,768644740597.12,691363412188.46,684898755570.72,767025741475.04,822538393591.45,786460035395.03,844459722152.99,877863284874.07,NaN
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,137244439121.37,87219300384.51,49840491178.15,68972769395.63,77792944471.95,69309110145.77,50241368243.63,65685435100.50,106713618735.43,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7074392884.05,6295845465.36,6682674062.96,7180768692.26,7878762812.20,7899741278.94,7717143395.26,9412034299.23,9429156201.96,NaN
262,"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43228585321.33,42444489460.94,31317824906.44,26842231204.80,21606161066.21,NaN,NaN,NaN,NaN,NaN
263,South Africa,8748596504.06,9225996313.29,9813996078.33,10854195662.66,11955995222.38,13068994777.63,14211394321.13,15821393677.77,17124793156.93,...,381198869776.11,346709790458.56,323585509674.48,381448814653.46,404158882459.02,388531226582.35,337619569570.51,419015636064.71,405869718462.34,NaN
264,Zambia,713000000.00,696285714.29,693142857.14,718714285.71,839428571.43,1082857142.86,1264285714.29,1368000000.00,1605857142.86,...,27141023558.08,21251216798.78,20958412538.31,25873601260.84,26311506435.06,23308667781.23,18110638619.37,22147649568.61,29784454055.94,NaN


In [71]:
gdp_country_list = pd.DataFrame(data=gdp_data['Country Name'].unique(), columns = ['gdp_country'])
gdp_country_list = gdp_country_list.sort_values(by='gdp_country', ignore_index= True)
gdp_country_list

,gdp_country
0,Afghanistan
1,Africa Eastern and Southern
2,Africa Western and Central
3,Albania
4,Algeria
...,...
261,West Bank and Gaza
262,World
263,"Yemen, Rep."
264,Zambia


## Inflation

In [14]:
# from fred, index 2015, 100
inflation_data = pd.read_csv('data/CPI-data-fred.csv')
inflation_data = inflation_data.rename(columns = {'DATE': 'Period', 'CPALCY01CAM661N':'cpi_val'})
inflation_data['Period'] = pd.to_datetime(inflation_data['Period'])

In [15]:
inflation_data

,Period,cpi_val
0,1988-01-01,56.93
1,1988-02-01,57.33
2,1988-03-01,57.49
3,1988-04-01,57.65
4,1988-05-01,57.97
...,...,...
424,2023-05-01,124.51
425,2023-06-01,125.07
426,2023-07-01,125.23
427,2023-08-01,126.03


# 1. Cleaning
---
---

### Merge 1605 and 0306

In [16]:
# concat function
def concat_dfs(df_list):
    df = pd.concat(df_list, axis = 0, ignore_index = True)
    return df

In [19]:
full_data = concat_dfs([df_1605_full,df_0306_full])


## Cleaning Steps

In [130]:
# perform cleaning set on combined data set
def cleaning_steps(df_name):
    # make lower case
    df_name['Commodity'] = df_name['Commodity'].str.lower()
    # keep only commodities that mention 'lobster'
    df_name = df_name[df_name['Commodity'].str.contains('lobster')].reset_index(drop=True)
    # drop extra columns
    df_name = df_name.drop(columns = ['Unit of measure'])
    # rename columns
    new_col_names = {'Value ($)': 'Value', 'Quantity': 'Quantity_kg'}
    df_name = df_name.rename(columns=new_col_names)
    # update period to datetime object
    df_name['Period'] = pd.to_datetime(df_name['Period'])
    return df_name



In [145]:
data_clean = cleaning_steps(full_data)
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059
1,2018-06-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2199419,48158
2,2014-06-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,1299983,38486
3,2014-10-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,151542,3475
4,2014-11-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,634072,22628
...,...,...,...,...,...,...,...
57380,2006-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,Hong Kong,NaN,6000,273
57381,2007-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,Japan,NaN,7200,408
57382,2007-03-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,"Korea, South",NaN,10080,572
57383,2006-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nunavut,Czechia,NaN,3000,136


### Adjust value to today's dollars

In [146]:

def inflation_adjustment(df_name, cpi_df):
    ''' adjust monetary features into today's dolars, index 2015 = 100'''
    df_name = df_name.merge(cpi_df, how='inner', on='Period')
    today_cpi = cpi_df.loc[cpi_df['Period'] == cpi_df['Period'].max(), 'cpi_val'].item()
    # create new column to adjust to today's dollars
    # return df_name
    df_name['Value-adjusted'] = (df_name['Value'] / df_name['cpi_val']) * today_cpi
    return df_name
    
    

In [147]:
data_clean = inflation_adjustment(data_clean, inflation_data)
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg,cpi_val,Value-adjusted
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,102.17,3023516.55
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,102.17,24808298.71
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,102.17,38330.44
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,102.17,121476.42
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,102.17,383038.25
...,...,...,...,...,...,...,...,...,...
57380,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,59.65,172082.30
57381,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,59.65,226273.05
57382,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,59.65,33862.36
57383,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,59.65,263598.02


## Country Name Cleaning

### Clean Country Name in export data first 
before running match

In [148]:
data_clean[["Country", 'Terminated']]= data_clean["Country"].str.split(r'(',expand=True)
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg,cpi_val,Value-adjusted,Terminated
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,102.17,3023516.55,None
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,102.17,24808298.71,None
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,102.17,38330.44,None
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,102.17,121476.42,None
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,102.17,383038.25,None
...,...,...,...,...,...,...,...,...,...,...
57380,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,59.65,172082.30,None
57381,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,59.65,226273.05,None
57382,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,59.65,33862.36,None
57383,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,59.65,263598.02,None


In [150]:
data_clean["Country"] = data_clean["Country"].str.strip()

In [151]:
export_country_list = pd.Series(data_clean['Country'].unique())
export_country_list = export_country_list.sort_values(ignore_index= True)
export_country_list

0              Afghanistan
1                  Albania
2      Antigua and Barbuda
3                Argentina
4                  Armenia
              ...         
155             Uzbekistan
156              Venezuela
157               Viet Nam
158           West Germany
159             Yugoslavia
Length: 160, dtype: object

In [54]:
# export clean full data for reuse

data_clean.to_csv('data/data_full_clean.csv')

### Match country names in gdp list to export country list

In [137]:
export_country_list.to_csv('data/export_countries.csv')

In [55]:
# filter GDP data by countries in export country list:
gdp_data_filtered = gdp_data[gdp_data['Country Name'].isin(export_country_list)].reset_index(drop=True)
gdp_data_filtered

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2791061452.51,2963128491.62,2983798882.68,3092178770.95,3276187709.50,3395793854.75,2610038938.55,3126019385.47,NaN,NaN
1,Afghanistan,537777811.11,548888895.56,546666677.78,751111191.11,800000044.44,1006666637.78,1399999966.67,1673333417.78,1373333366.67,...,20550582746.84,19998143635.87,18019554403.45,18896353155.88,18418860354.42,18904502222.21,20143451705.75,14583135236.57,NaN,NaN
2,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13228147516.12,11386853143.15,11861199830.84,13019726211.74,15156424061.98,15401826080.52,15162734205.25,17930565118.82,18882095517.88,NaN
3,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,414105366752.89,370275469571.14,369255326235.53,390516804029.95,427049432157.93,417989721742.68,349473015330.16,415021590687.54,507534921715.45,NaN
4,Argentina,NaN,NaN,24450604877.61,18272123664.47,25605249381.76,28344705966.64,28630474727.90,24256667553.26,26436857247.50,...,526319673731.64,594749285413.21,557532317363.45,643628396190.39,524819898586.91,447754686715.08,385540406815.60,487227125385.64,632770284408.51,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,133503867181.97,91030968761.42,93355868977.66,112090503817.40,130891086689.52,153883045524.93,156617719786.25,199765856764.50,160502739236.10,NaN
116,Uruguay,1242289239.20,1547388781.43,1710004407.23,1539681490.78,1975701816.47,1890767155.53,1809185093.55,1597713469.10,1593674184.56,...,61496186973.90,57680328702.24,57480788380.31,65006047680.32,65203071110.60,62048585618.50,53666908479.56,61412268248.95,71177146197.50,NaN
117,United States,543300000000.00,563300000000.00,605100000000.00,638600000000.00,685800000000.00,743700000000.00,815000000000.00,861700000000.00,942500000000.00,...,17550680174000.00,18206020741000.00,18695110842000.00,19477336549000.00,20533057312000.00,21380976119000.00,21060473613000.00,23315080560000.00,25462700000000.00,NaN
118,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80845385816.21,86196264741.94,86138288633.22,62081322741.12,52870108217.56,60283503705.39,60224701295.80,69600614988.60,80391853884.76,NaN


In [66]:
# unmatched countries 
unmatched = list(data_clean.loc[~data_clean['Country'].isin(gdp_data_filtered['Country Name']), 'Country'].unique())
unmatched = pd.DataFrame(data=unmatched, columns=['unmatched_export_countries'])

In [67]:
unmatched

,unmatched_export_countries
0,Hong Kong
1,"Korea, South"
2,Taiwan
3,Gambia
4,Macao
5,Christmas Island
6,Türkiye
7,United States Minor Outlying Islands
8,"South Africa, Republic of"
9,"Moldova, Republic of"


In [ ]:
# for unmatched in unmatched_countries:

In [57]:
gdp_data.loc[gdp_data['Country Name'].str.contains('Hong'), 'Country Name'].item()


'Hong Kong SAR, China'

In [72]:
pd.concat([unmatched, gdp_country_list], axis = 1)

,unmatched_export_countries,gdp_country
0,Hong Kong,Afghanistan
1,"Korea, South",Africa Eastern and Southern
2,Taiwan,Africa Western and Central
3,Gambia,Albania
4,Macao,Algeria
...,...,...
261,NaN,West Bank and Gaza
262,NaN,World
263,NaN,"Yemen, Rep."
264,NaN,Zambia


In [135]:
from thefuzz import process
import pycountry

In [136]:
# Function to convert country names to standardized names using pycountry
def standardize_country_names(name):
  try:
    # Try to match the official country name
    return pycountry.countries.search_fuzzy(name)[0].name
  except:
    return name  # Return the original name if no match is found

In [127]:
# def country_match(countries_to_match, country_options):
result_df = pd.DataFrame(columns = ['country_to_match','best_match', 'all_matches'] )
for country_index in range(len(unmatched)):
        country_to_match = unmatched.iloc[country_index,0]
        collection = gdp_country_list.iloc[:,0].to_list()
        best_matches = process.extract(country_to_match, collection, scorer=fuzz.ratio)
        new_row = {'country_to_match': country_to_match, 'best_match':best_matches[0], 'all_matches':str(best_matches)}
        # print(pd.DataFrame(data = [new_row]))
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)

        # result_df = result_df.append(country_to_match, best_matches[0], best_matches, ignore_index = True)
result_df

,country_to_match,best_match,all_matches
0,Hong Kong,"(Hong Kong SAR, China, 62)","[('Hong Kong SAR, China', 62), ('Mongolia', 47..."
1,"Korea, South","(Korea, Rep., 64)","[('Korea, Rep.', 64), ('Lesotho', 53), ('Afric..."
2,Taiwan,"(Thailand, 71)","[('Thailand', 71), ('Tajikistan', 62), ('Botsw..."
3,Gambia,"(Zambia, 83)","[('Zambia', 83), ('Namibia', 77), ('Cambodia',..."
4,Macao,"(Monaco, 73)","[('Monaco', 73), ('Curacao', 67), ('Jamaica', ..."
5,Christmas Island,"(Cayman Islands, 67)","[('Cayman Islands', 67), ('Channel Islands', 6..."
6,Türkiye,"(Turkiye, 86)","[('Turkiye', 86), ('Austria', 43), ('Eritrea',..."
7,United States Minor Outlying Islands,"(British Virgin Islands, 55)","[('British Virgin Islands', 55), ('Northern Ma..."
8,"South Africa, Republic of","(Central African Republic, 69)","[('Central African Republic', 69), ('South Afr..."
9,"Moldova, Republic of","(Slovak Republic, 74)","[('Slovak Republic', 74), ('Dominican Republic..."


In [97]:
collection = gdp_country_list.iloc[:,0].to_list()
# unmatched.iloc[0,0]
# collection


print('best match for:',unmatched.iloc[0,0],'is: \n',process.extract(unmatched.iloc[0,0], collection, scorer=fuzz.ratio))

best match for: Hong Kong is: 
 [('Hong Kong SAR, China', 62), ('Mongolia', 47), ('Togo', 46), ('European Union', 43), ('Gabon', 43)]


In [ ]:
{fuzz.token_set_ratio(unmatched_export_countries, gdp_country)}")

"""
Token sort ratio similarity score: 100

# 2. EDA on Full Data Set
---
---

In [ ]:
# surpress scientific notation
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
full_data['Province'].value_counts()

In [ ]:
# null values in state are for countries other than US
full_data.info()

In [ ]:
full_data.loc[(full_data['Country'] == 'United States'), 'State'].value_counts()

In [ ]:
exports = data_clean.groupby('Country').mean('Value-adjusted')
exports.sort_values(by='Quantity_kg', ascending=False).head(20)


In [ ]:
# store in a list
top_countries = list(exports.sort_values(by='Quantity_kg', ascending=False).head(20).index)
top_countries

In [ ]:
data_clean.groupby('Country').mean('Quantity_kg').sort_values(by='Quantity_kg', ascending = False)

In [ ]:
pd.DataFrame(data_clean['Country'].value_counts())

In [ ]:
pd.DataFrame(data_clean['Commodity'].value_counts())

In [ ]:
filtered_df = data_clean.loc[data_clean['Country'].isin(top_countries),:]
filtered_df

In [ ]:
filtered_df

In [ ]:
cols = ['Period', 'Country', 'Value($)']
filtered_df = filtered_df[['Period','Commodity', 'Country', 'Quantity', 'Value ($)']]


In [ ]:
data

In [ ]:
sns.scatterplot(x=data['Period'], y=data['Value ($)'], alpha = 0.2)


# bin

dont need... got relevant HS codes another way

In [ ]:
hs_8 = pd.read_csv('data/codes_hs8.csv', sep = ',',skiprows = 1, engine='python', skipfooter = 5, index_col=False)
# hs_8 = pd.read_csv('codes_hs8.csv', sep = ',', skipfooter = 5)


In [ ]:
pd.set_option('display.max_colwidth', None)
hs_8['Commodity'] = hs_8['Commodity'].str.lower()

In [ ]:
hs_lobster = hs_8.loc[hs_8['Commodity'].str.contains('lobster'),:]

In [ ]:
hs_lobster['Commodity'].unique()

### Testing univariate ARIMA

With only data from 1 country, for 1 commodity, using Quantity.


In [ ]:
# find country with most complete data accross time. most countries have great data since 2017

In [ ]:
data

In [ ]:
codes = ['0306.22.10', '0306.32.10']



In [ ]:
arima_test = filtered_df[filtered_df['Commodity'].str[:10].isin(codes)].sort_values(by='Period')
arima_test.groupby('Country').count().sort_values(by='Period', ascending = False)
# arima_test

In [ ]:
# filter for one country --- belgium as example
arima_test = arima_test.loc[arima_test['Country'] == 'Belgium']

In [ ]:
# drop cextra columns
arima_test.drop(columns=['Country', 'Value ($)'], inplace = True)
arima_test

In [ ]:
# count records by year, check that there is no overlap
arima_test.groupby(arima_test['Period'].dt.year).count()

In [ ]:
# arima_test = arima_test.drop(columns=['Commodity'])
arima_test = arima_test.reset_index(drop=True)
arima_test

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(x = arima_test['Period'], y=arima_test['Quantity'])

In [ ]:
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot

def parser(x):
 return datetime.strptime('190'+x, '%Y-%m')

series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
autocorrelation_plot(series)
pyplot.show()

In [ ]:
japan_data = japan_data[['Period','Quantity']]
japan_data

In [ ]:

japan_data = japan_data.groupby('Period').sum(['Quantity', 'Value ($)']).reset_index(drop=False)
japan_data['Value per kg'] = japan_data['Value ($)'] / japan_data['Quantity']
japan_data

In [ ]:
japan_data['Year'] = japan_data['Period'].dt.year
japan_data



### other

In [ ]:
japan_data_recent = japan_data.loc[japan_data['Period'] > '2020', :].reset_index(drop = True)
japan_data_recent

In [ ]:
sns.lineplot(x=japan_data_recent['Year'], y=japan_data_recent['Quantity'])

In [ ]:
data

In [ ]:
sns.lineplot(x=data['Period'], y=data['Value ($)'])